In [ ]:
# Install necessary packages
!pip install -q sentence-transformers faiss-cpu

# Imports
import pandas as pd
import random
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.utils import shuffle
import torch
from torch.nn.functional import cosine_similarity

# Load your dataset
df = pd.read_csv("training_data.csv")

# Create training pairs
def create_training_pairs(df):
    queries, positives, negatives = [], [], []
    for i in range(len(df)):
        query = df.iloc[i]['tags']
        pos = df.iloc[i]['tags']
        neg_idx = random.choice(df[df['rating'] < 3.0].index)
        negative = df.loc[neg_idx]['tags']
        queries.append(query)
        positives.append(pos)
        negatives.append(negative)
    return queries, positives, negatives

queries, positives, negatives = create_training_pairs(df)

# Initialize Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Prepare training data
train_examples = [
    InputExample(texts=[q, p], label=1.0) for q, p in zip(queries, positives)
] + [
    InputExample(texts=[q, n], label=0.0) for q, n in zip(queries, negatives)
]

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True
)

# Save the model in a folder
model.save("nlp-recipe-finder")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: harsh-t-verma (harsh-t-verma-university-at-buffalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.015800
1000,0.007500
1500,0.006800
2000,0.006300
2500,0.006000
3000,0.006000
3500,0.005600
4000,0.005500
4500,0.005300
5000,0.005200


In [ ]:

# Encode recipes for retrieval
recipe_embeddings = model.encode(df['tags'].tolist(), convert_to_tensor=True)

# Sample query
query = "potato mushroom stew dinner easy-to-make"
query_embedding = model.encode(query, convert_to_tensor=True)

# Similarity ranking
sims = cosine_similarity(query_embedding, recipe_embeddings)
top_k = torch.topk(sims, k=10)
top_indices = top_k.indices.tolist()

# Return top results sorted by rating
results = df.iloc[top_indices]
results = results.sort_values(by='rating', ascending=False)
print(results[['recipe_id', 'name', 'tags', 'rating']])

        recipe_id                               name  \
204785      24396   crispy mushroom   onion potatoes   
140942     385970        browned leeks and mushrooms   
3258        12100           amazing stuffed  shrooms   
190707      58676         chicken and mushroom saute   
43913       51489                scalloped mushrooms   
130798     315763  gilroy marinated garlic mushrooms   
29209       44786             fresh creamy mushrooms   
100208     192604         senior s sauteed mushrooms   
192967     360855    the best beef stroganoff recipe   
176195      47015      sauteed mushrooms with garlic   

                                                     tags  rating  
204785  weeknight time-to-make course main-ingredient ...    5.00  
140942  30-minutes-or-less time-to-make main-ingredien...    5.00  
3258    30-minutes-or-less time-to-make course prepara...    4.50  
190707  bacon 30-minutes-or-less time-to-make course m...    4.50  
43913   60-minutes-or-less time-to-make cou

In [ ]:
!zip -r nlp-recipe-finder.zip nlp-recipe-finder/

  adding: nlp-recipe-finder/ (stored 0%)
  adding: nlp-recipe-finder/model.safetensors (deflated 8%)
  adding: nlp-recipe-finder/1_Pooling/ (stored 0%)
  adding: nlp-recipe-finder/1_Pooling/config.json (deflated 57%)
  adding: nlp-recipe-finder/vocab.txt (deflated 53%)
  adding: nlp-recipe-finder/tokenizer.json (deflated 71%)
  adding: nlp-recipe-finder/README.md (deflated 74%)
  adding: nlp-recipe-finder/modules.json (deflated 62%)
  adding: nlp-recipe-finder/config.json (deflated 48%)
  adding: nlp-recipe-finder/2_Normalize/ (stored 0%)
  adding: nlp-recipe-finder/sentence_bert_config.json (deflated 4%)
  adding: nlp-recipe-finder/special_tokens_map.json (deflated 80%)
  adding: nlp-recipe-finder/tokenizer_config.json (deflated 73%)
  adding: nlp-recipe-finder/config_sentence_transformers.json (deflated 34%)
